# init

In [1]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping

import Geometry
from Geometry import Point
from camera import Camera
import draw
import vec
from inputs import input_update

# reload modules

In [220]:

importlib.reload(Clipping)
importlib.reload(Geometry)
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
importlib.reload(sys.modules['opengl_draw_2d'])
importlib.reload(sys.modules['opengl_draw_3d'])
import Geometry

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
import draw



# build shapes

In [236]:
from colors import *
    
d = 4 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = Geometry.build_cube(d)

#draw a pair of cubes for each axis
for face, color in zip(cube_shape.faces,tess_face_colors):
    #face.color = color
    face.color = GREEN
shapes = []
r = 4
for i in range(d):
    cube = cube_shape.copy()
    cube.update(pos = r*np.eye(d)[i])
    shapes.append(cube)
    cube = cube_shape.copy()
    cube.update(pos = -r*np.eye(d)[i])
    shapes.append(cube)
#shapes = shapes[:2] + shapes[-4:] 
#shapes = shapes[:2] + shapes[-2:]
#shapes.pop(2) #removes top cube
#cube_shape.transparent = True
    
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)
    
floor_lines = Geometry.build_floor(d,1,16,-5)

## other shapes

In [124]:
d = 3
sphere_shape = Geometry.build_sphere_3d(r=1,phi_pts=10,theta_pts=5)
#shapes = [sphere_shape]
shapes.append(sphere_shape)

In [244]:
d = 4
m = 12; n = 12
cylinder_shape = Geometry.build_duocylinder(rs=[1,1],axes=[[0,1],[2,3]],n_circ_pts=[m,n])
#cylinder_shape.transparent = True
for i,face in enumerate(cylinder_shape.faces[:m]):
    face.color = ORANGE
    #face.color = (np.cos(2*np.pi*i/m)**2,np.sin(2*np.pi*i/m)**2,1)
    #face.color = ((i+1)%2,(i+1)%2,i%2)
for i,face in enumerate(cylinder_shape.faces[m:]):
    face.color = CYAN
    #face.color = (1,np.cos(2*np.pi*i/m)**2,np.sin(2*np.pi*i/m)**2)
    #face.color = ((i+1)%2,i%2,(i+1)%2)
    #face.color = ((i+1)%2,(i+1)%2,i%2)
shapes = [cylinder_shape]
#shapes.append(cylinder_shape)

# main loop

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Clipping happens in 2

Todo: 
- fix target shape clipping
- speed up /generalize floor drawing
- speed up clipping
- find a way to make the center obvious wrt objects in the view. does rotation help or hurt? how can one navigate with a rotating viewport?
- other points of reference at infinity
- a floor and ceiling
- implement cubinders, spherinders
- generalize face fuzz to all convex faces, invariant to edge order

In [24]:
import colors
import vec
def test_points(camera,pos):
    points = [ vec.Vec([np.cos(th),np.sin(th),0,0]) + pos for th in np.linspace(0,2*np.pi,200)]
    draw.draw_points(camera,points,colors.GREEN)

In [ ]:
#does not draw before input / does not update camera properly?

t = 0
#size = width, height = 1100, 700
size = width, height = 1600, 900

pygame.init()

if d == 3:
    draw.init(d,size,focal=6,n_fuzz_points = 200)
    draw.view_radius = 7
    draw_view_height = 7
    draw.view_boundary = 'none'
    camera = Camera(pos = Point([0,0,-10]))
    camera.target_shape = Geometry.build_3d_target()
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
    camera.look_at(vec.zero_vec(d))
    #camera.look_at(vec.zero_vec(d))
    #camera = Camera(pos = Point([0,0,0]))
if d == 4:
    draw.init(d,size,stereo=False,n_fuzz_points = 200)
    draw.view_boundary = 'none'
    if draw.stereo:
        draw.view_radius = 4
    else:
        draw.view_radius = 6
        draw.view_height = 4
    #camera = Camera(pos = Point([0,0,0,0]))
    draw.face_scales = [0.9]
    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    #camera = Camera(pos = Point([0,0,0,0]))
    camera = Camera(pos = vec.one_hot(d,-1)*(-5))
    camera.look_at(vec.zero_vec(d))
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
draw.clipping = True
quit = False
redraw = True
timer = time.time()
t0 = time.time()
frame = 0
try:
    while not quit:

        if redraw:
            draw.draw(camera,shapes+ [camera.target_shape])
            draw.clip_and_draw_lines(floor_lines,camera,(0.2,0.2,0.2),None,shapes)
            #test_points(camera,shapes[0].pos)
            #draw.draw_wireframe(camera,cylinder_shape,GRAY)
            #draw.draw_normals(camera,shapes[0],WHITE)
            pygame.display.flip()
            redraw = False
            
        #this is here so that we draw on the first frame
        redraw, quit = input_update(camera, shapes)
        #camera.update_rot_matrix(0, 2, camera.ang_speed/2)
        #cylinder_shape.rotate(0,2,0.01)
        #cylinder_shape.rotate(1,3,0.02)
        redraw = True #redraw every loop. useful for assessing framerate


        delta_t = (time.time() - timer)
        if delta_t > 2:
        #if False:
            #print('redraw:',redraw)
            try:
                #fps = 30/(time.time() - timer)
                fps = frame/delta_t
            except:
                fps = 0
            #if fps < 60:
            print(fps,'fps')
            timer = time.time()
            frame = 0
        frame = frame+1
except:
    pygame.quit()
    print('game terminated')
    raise
pygame.quit()
print('game terminated')

init GL
29.455940318940094 fps
28.40121721354299 fps
31.20480342145999 fps
44.486588170351645 fps
44.55116415204789 fps
32.14190505001965 fps
24.49708264046218 fps
26.245926061658498 fps
41.09737486114046 fps
40.59477370196799 fps
46.03820246361513 fps
46.49907984170273 fps
46.49135416597141 fps
44.05116544228582 fps
47.498856214409436 fps
38.491162541025396 fps
46.52030109822789 fps
46.02725620657181 fps
47.02863767168994 fps
45.9940016939815 fps
45.998338520935555 fps
46.49995011096585 fps
46.49544945192336 fps
46.51855564546254 fps
46.99826318483504 fps
47.498171101896055 fps
47.50335187087874 fps
47.007610238028164 fps
47.49951303504455 fps
46.99891307483476 fps
47.525908467700816 fps
47.52309823837228 fps
47.99824339619605 fps
46.515163913731456 fps
48.017286093678486 fps
44.99838536810925 fps
47.99880984396368 fps
47.5072807738912 fps
46.99460508767373 fps
46.515855391710964 fps
46.99644807004018 fps
47.00895232481784 fps
45.99827272225529 fps
47.01213038133545 fps
46.53436540896

In [188]:
pygame.quit()

# workspace

In [79]:
import vec
vec.norm(Vec([-5.55299282,  3.03433562]))

6.327947685764504

In [7]:
def check_normals(shape):
    def check_face(face):
        vertis = face.get_verts(shape)
        verts = [shape.verts_ref[vi] for vi in vertis]
        center = face.center
        dots = [vec.dot(v-face.center,face.normal_ref) for v in verts]
        return np.all(vec.isclose(dots,vec.zero_vec(len(dots))))
    return [check_face(face) for face in shape.faces]

In [119]:
from numpy.linalg import svd
def test(shape,face):
        verts_in_face = [shape.verts[vi] for vi in face.get_verts(shape)]
        rel_verts = [v - face.center for v in verts_in_face]
        #find vector perpendicular to all vertices
        return svd(np.array(rel_verts))
print(test(cylinder_shape,cylinder_shape.faces[5])[-1][-1])
print(cylinder_shape.faces[5].normal_ref)

[-0.         -0.         -0.70710678 -0.70710678]
[-5.65986790e-17  0.00000000e+00  7.07106781e-01  7.07106781e-01]


In [141]:
cylinder_shape.check_normals()

True

In [142]:
Clipping.clip_line_plane(Geometry.Line(Vec([0,0]),Vec([1,-3])),Geometry.HyperPlane(vec.one_hot(2,1),-2))

Line([0 0],[ 0.66666667 -2.        ])

In [52]:
from vec import Vec
test =Clipping.sphere_line_intersect(Geometry.Line(Vec([2,0]),Vec([-2,0])),4,return_ts=True)
print(test)

([array([4., 0.]), array([-4.,  0.])], (6.0, -2.0))


In [64]:
n_circ_pts = [3,4]
print([(j+i*n_circ_pts[1],(j+1)%n_circ_pts[1]+i*n_circ_pts[1])
        for i,j in itertools.product(range(n_circ_pts[0]),range(n_circ_pts[1]))])
[(j+i*n_circ_pts[1],j + ((i+1)%n_circ_pts[0])*n_circ_pts[1])
        for i,j in itertools.product(range(n_circ_pts[0]),range(n_circ_pts[1]))]

[(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (9, 10), (10, 11), (11, 8)]


[(0, 4),
 (1, 5),
 (2, 6),
 (3, 7),
 (4, 8),
 (5, 9),
 (6, 10),
 (7, 11),
 (8, 0),
 (9, 1),
 (10, 2),
 (11, 3)]

In [58]:
[(i,j) for i,j in itertools.product(range(n_circ_pts[0]),range(n_circ_pts[1]))]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [67]:
shapes[0].update(angles=[0,time.time()])

[[ 1.          0.          0.          0.        ]
 [ 0.         -0.97774833  0.         -0.20978138]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.20978138  0.         -0.97774833]]


In [68]:
time.time()

1556095332.9433315